In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

/kaggle/input/training-dataset-from-covid-19-week-1-forecasting/train-3.csv
/kaggle/input/covid19-global-forecasting-week-2/submission.csv
/kaggle/input/covid19-global-forecasting-week-2/test.csv
/kaggle/input/covid19-global-forecasting-week-2/train.csv
/kaggle/input/population-by-country-2020/population_by_country_2020.csv


In [2]:
import pandas as pd
#import plotly.express as px
#import matplotlib.pyplot as plt
#import plotly.graph_objects as go
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
import numpy as np
from sklearn.metrics import mean_squared_log_error
from sklearn.tree import DecisionTreeRegressor
from google.cloud import bigquery
from scipy.spatial.distance import cdist
from sklearn.preprocessing import LabelEncoder

In [3]:
train_df=pd.read_csv("../input/covid19-global-forecasting-week-2/train.csv")
test_df=pd.read_csv("../input/covid19-global-forecasting-week-2/test.csv")

In [4]:
train_df.head()

,Id,Province_State,Country_Region,Date,ConfirmedCases,Fatalities
0,1,NaN,Afghanistan,2020-01-22,0.0,0.0
1,2,NaN,Afghanistan,2020-01-23,0.0,0.0
2,3,NaN,Afghanistan,2020-01-24,0.0,0.0
3,4,NaN,Afghanistan,2020-01-25,0.0,0.0
4,5,NaN,Afghanistan,2020-01-26,0.0,0.0


In [5]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20580 entries, 0 to 20579
Data columns (total 6 columns):
Id                20580 non-null int64
Province_State    8750 non-null object
Country_Region    20580 non-null object
Date              20580 non-null object
ConfirmedCases    20580 non-null float64
Fatalities        20580 non-null float64
dtypes: float64(2), int64(1), object(3)
memory usage: 964.8+ KB


In [6]:
test_df.head()

,ForecastId,Province_State,Country_Region,Date
0,1,NaN,Afghanistan,2020-03-19
1,2,NaN,Afghanistan,2020-03-20
2,3,NaN,Afghanistan,2020-03-21
3,4,NaN,Afghanistan,2020-03-22
4,5,NaN,Afghanistan,2020-03-23


In [7]:
print("Min train date: ",train_df["Date"].min())
print("Max train date: ",train_df["Date"].max())
print("Min test date: ",test_df["Date"].min())
print("Max test date: ",test_df["Date"].max())

Min train date:  2020-01-22
Max train date:  2020-03-31
Min test date:  2020-03-19
Max test date:  2020-04-30


In [8]:
train_df=train_df[train_df["Date"]<"2020-03-19"]

In [9]:
train_df.isnull().sum()

Id                   0
Province_State    9633
Country_Region       0
Date                 0
ConfirmedCases       0
Fatalities           0
dtype: int64

In [10]:
test_df.isnull().sum()

ForecastId           0
Province_State    7267
Country_Region       0
Date                 0
dtype: int64

In [11]:
pop_info = pd.read_csv("../input/population-by-country-2020/population_by_country_2020.csv")

In [12]:
pop_info.rename(columns={'Density (P/Km²)': 'Density'}, inplace=True)

In [13]:
pop_info.columns

Index(['Country (or dependency)', 'Population (2020)', 'Yearly Change',
       'Net Change', 'Density', 'Land Area (Km²)', 'Migrants (net)',
       'Fert. Rate', 'Med. Age', 'Urban Pop %', 'World Share'],
      dtype='object')

In [14]:
country_lookup=pop_info[["Country (or dependency)","Population (2020)","Density"]]

In [15]:
pd.DataFrame.from_dict(country_lookup)
train_df_pop=pd.merge(train_df, country_lookup, how='left', left_on='Country_Region', right_on='Country (or dependency)')

In [16]:
train_df_pop.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 16758 entries, 0 to 16757
Data columns (total 9 columns):
Id                         16758 non-null int64
Province_State             7125 non-null object
Country_Region             16758 non-null object
Date                       16758 non-null object
ConfirmedCases             16758 non-null float64
Fatalities                 16758 non-null float64
Country (or dependency)    13167 non-null object
Population (2020)          13167 non-null float64
Density                    13167 non-null float64
dtypes: float64(4), int64(1), object(4)
memory usage: 1.3+ MB


In [17]:
train_df_pop.loc[train_df_pop["Country_Region"]=="US", ["Population (2020)"]]=331002651 #United Sates
train_df_pop.loc[train_df_pop["Country_Region"]=="US", ["Density"]]=36
train_df_pop.loc[train_df_pop["Country_Region"]=="Korea, South", ["Population (2020)"]]=51269185 #South Korea
train_df_pop.loc[train_df_pop["Country_Region"]=="Korea, South", ["Density"]]=527
train_df_pop.loc[train_df_pop["Country_Region"]=="Czechia", ["Population (2020)"]]=10708981 #Czech Republic
train_df_pop.loc[train_df_pop["Country_Region"]=="Czechia", ["Density"]]=139
train_df_pop.loc[train_df_pop["Country_Region"]=="Taiwan*", ["Population (2020)"]]=23816775 #Taiwan
train_df_pop.loc[train_df_pop["Country_Region"]=="Taiwan*", ["Density"]]=673
train_df_pop.loc[train_df_pop["Country_Region"]=="Congo (Kinshasa)", ["Population (2020)"]]=89561403 #DR Congo
train_df_pop.loc[train_df_pop["Country_Region"]=="Congo (Kinshasa)", ["Density"]]=40
train_df_pop.loc[train_df_pop["Country_Region"]=="Congo (Brazzaville)", ["Population (2020)"]]=5518087 #Congo
train_df_pop.loc[train_df_pop["Country_Region"]=="Congo (Brazzaville)", ["Density"]]=16
train_df_pop.loc[train_df_pop["Country_Region"]=="Cote d'Ivoire", ["Population (2020)"]]=26378274 #CÃ´te d'Ivoire
train_df_pop.loc[train_df_pop["Country_Region"]=="Cote d'Ivoire", ["Density"]]=83
train_df_pop.loc[train_df_pop["Country_Region"]=="Saint Kitts and Nevis", ["Population (2020)"]]=53199 #Saint Kitts & Nevis
train_df_pop.loc[train_df_pop["Country_Region"]=="Saint Kitts and Nevis", ["Density"]]=205
train_df_pop.loc[train_df_pop["Country_Region"]=="Saint Vincent and the Grenadines", ["Population (2020)"]]=110940 #St. Vincent & Grenadines
train_df_pop.loc[train_df_pop["Country_Region"]=="Saint Vincent and the Grenadines", ["Density"]]=284
train_df_pop.loc[train_df_pop["Country_Region"]=="Diamond Princess", ["Population (2020)"]]=3770 #Population and density are same since it is a cruise ship
train_df_pop.loc[train_df_pop["Country_Region"]=="Diamond Princess", ["Density"]]=3770

In [18]:
test_df_pop=pd.merge(test_df, country_lookup, how='left', left_on='Country_Region', right_on='Country (or dependency)')

In [19]:
test_df_pop.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 12642 entries, 0 to 12641
Data columns (total 7 columns):
ForecastId                 12642 non-null int64
Province_State             5375 non-null object
Country_Region             12642 non-null object
Date                       12642 non-null object
Country (or dependency)    9933 non-null object
Population (2020)          9933 non-null float64
Density                    9933 non-null float64
dtypes: float64(2), int64(1), object(4)
memory usage: 790.1+ KB


In [20]:
test_df_pop.loc[test_df_pop["Country_Region"]=="US", ["Population (2020)"]]=331002651 #United Sates
test_df_pop.loc[test_df_pop["Country_Region"]=="US", ["Density"]]=36
test_df_pop.loc[test_df_pop["Country_Region"]=="Korea, South", ["Population (2020)"]]=51269185 #South Korea
test_df_pop.loc[test_df_pop["Country_Region"]=="Korea, South", ["Density"]]=527
test_df_pop.loc[test_df_pop["Country_Region"]=="Czechia", ["Population (2020)"]]=10708981 #Czech Republic
test_df_pop.loc[test_df_pop["Country_Region"]=="Czechia", ["Density"]]=139
test_df_pop.loc[test_df_pop["Country_Region"]=="Taiwan*", ["Population (2020)"]]=23816775 #Taiwan
test_df_pop.loc[test_df_pop["Country_Region"]=="Taiwan*", ["Density"]]=673
test_df_pop.loc[test_df_pop["Country_Region"]=="Congo (Kinshasa)", ["Population (2020)"]]=89561403 #DR Congo
test_df_pop.loc[test_df_pop["Country_Region"]=="Congo (Kinshasa)", ["Density"]]=40
test_df_pop.loc[test_df_pop["Country_Region"]=="Congo (Brazzaville)", ["Population (2020)"]]=5518087 #Congo
test_df_pop.loc[test_df_pop["Country_Region"]=="Congo (Brazzaville)", ["Density"]]=16
test_df_pop.loc[test_df_pop["Country_Region"]=="Cote d'Ivoire", ["Population (2020)"]]=26378274 #CÃ´te d'Ivoire
test_df_pop.loc[test_df_pop["Country_Region"]=="Cote d'Ivoire", ["Density"]]=83
test_df_pop.loc[test_df_pop["Country_Region"]=="Saint Kitts and Nevis", ["Population (2020)"]]=53199 #Saint Kitts & Nevis
test_df_pop.loc[test_df_pop["Country_Region"]=="Saint Kitts and Nevis", ["Density"]]=205
test_df_pop.loc[test_df_pop["Country_Region"]=="Saint Vincent and the Grenadines", ["Population (2020)"]]=110940 #St. Vincent & Grenadines
test_df_pop.loc[test_df_pop["Country_Region"]=="Saint Vincent and the Grenadines", ["Density"]]=284
test_df_pop.loc[test_df_pop["Country_Region"]=="Diamond Princess", ["Population (2020)"]]=3770 #Population and density are same since it is a cruise ship
test_df_pop.loc[test_df_pop["Country_Region"]=="Diamond Princess", ["Density"]]=3770

In [21]:
train_df_pop.drop("Country (or dependency)", axis=1, inplace=True)

In [22]:
test_df_pop.drop("Country (or dependency)", axis=1, inplace=True)

In [23]:
%load_ext google.cloud.bigquery

In [24]:
%%time
client = bigquery.Client()
dataset_ref = client.dataset("noaa_gsod", project="bigquery-public-data")
dataset = client.get_dataset(dataset_ref)

tables = list(client.list_tables(dataset))

table_ref = dataset_ref.table("stations")
table = client.get_table(table_ref)
stations_df = client.list_rows(table).to_dataframe()

table_ref = dataset_ref.table("gsod2020")
table = client.get_table(table_ref)
twenty_twenty_df = client.list_rows(table).to_dataframe()

stations_df['STN'] = stations_df['usaf'] + '-' + stations_df['wban']
twenty_twenty_df['STN'] = twenty_twenty_df['stn'] + '-' + twenty_twenty_df['wban']

cols_1 = ['STN', 'mo', 'da', 'temp', 'min', 'max', 'stp', 'wdsp', 'prcp', 'fog']
cols_2 = ['STN', 'country', 'state', 'call', 'lat', 'lon', 'elev']
weather_df = twenty_twenty_df[cols_1].join(stations_df[cols_2].set_index('STN'), on='STN')



Using Kaggle's public dataset BigQuery integration.
CPU times: user 2min 26s, sys: 5.61 s, total: 2min 31s
Wall time: 6min


In [25]:
weather_df["da"]=weather_df["da"].astype(int)

In [26]:
weather_df['day_from_jan_first'] = weather_df["da"] + 31*(weather_df["mo"]=='02') + 60*(weather_df["mo"]=='03') + 91*(weather_df["mo"]=='04')  
                              

In [27]:
train_wk1=pd.read_csv("../input/training-dataset-from-covid-19-week-1-forecasting/train-3.csv")


In [28]:
train_wk1['country+province'] = train_wk1['Country/Region'].fillna('') + '-' + train_wk1['Province/State'].fillna('')
train_df_pop['country+province'] = train_df_pop['Country_Region'].fillna('') + '-' + train_df_pop['Province_State'].fillna('')
test_df_pop['country+province'] = test_df_pop['Country_Region'].fillna('') + '-' + test_df_pop['Province_State'].fillna('')

Below code to join with weather data has been incorporated from this link
https://www.kaggle.com/davidbnn92/weather-data 

In [29]:
df = train_wk1.groupby('country+province')[['Lat', 'Long']].mean()

In [30]:
df.loc['United Kingdom-'] = df.loc['United Kingdom-United Kingdom']
df.loc['Diamond Princess-'] = df.loc['Cruise Ship-Diamond Princess']
df.loc['Denmark-'] = df.loc['Denmark-Denmark']
df.loc['France-'] = df.loc['France-France']
df.loc['Gambia-'] = df.loc['Gambia, The-']
df.loc['Netherlands-'] = df.loc['Netherlands-Netherlands']
df.loc['Dominica-'] = (15.3, -61.383333)
df.loc['Angola-'] = (-8.830833, 13.245)
df.loc['Bahamas-'] = (25.066667, -77.333333)
df.loc['Belize-'] = (17.498611, -88.188611)
df.loc['Cabo Verde-'] = (14.916667, -23.516667)
df.loc['Chad-'] = (12.134722, 15.055833)
df.loc['Denmark-Greenland'] = (64.181389, -51.694167)
df.loc['El Salvador-'] = (13.698889, -89.191389)
df.loc['Eritrea-'] = (15.322778, 38.925)
df.loc['Fiji-'] = (-18.166667, 178.45)
df.loc['France-Martinique'] = (14.666667, -61)
df.loc['France-New Caledonia'] = (-22.2758, 166.458)
df.loc['Grenada-'] = (12.05, -61.75)
df.loc['Guinea-Bissau-'] = (11.85, -15.566667)
df.loc['Haiti-'] = (18.533333, -72.333333)
df.loc['Laos-'] = (17.966667, 102.6)
df.loc['Libya-'] = (32.887222, 13.191389)
df.loc['Madagascar-'] = (-18.933333, 47.516667)
df.loc['Mali-'] = (12.639167, -8.002778)
df.loc['Mozambique-'] = (-25.966667, 32.583333)
df.loc['Netherlands-Sint Maarten'] = (18.052778, -63.0425)
df.loc['Nicaragua-'] = (12.136389, -86.251389)
df.loc['Niger-'] = (13.511667, 2.125278)
df.loc['Papua New Guinea-'] = (-9.478889, 147.149444)
df.loc['Saint Kitts and Nevis-'] = (17.3, -62.733333)
df.loc['Syria-'] = (33.513056, 36.291944)
df.loc['Timor-Leste-'] = (-8.566667, 125.566667)
df.loc['Uganda-'] = (0.313611, 32.581111)
df.loc['Zimbabwe-'] = (-17.829167, 31.052222)
df.loc['United Kingdom-Bermuda'] = (32.293, -64.782)
df.loc['United Kingdom-Isle of Man'] = (54.145, -4.482)

In [31]:
train_df_pop['Lat'] = train_df_pop['country+province'].apply(lambda x: df.loc[x, 'Lat'])
train_df_pop['Long'] = train_df_pop['country+province'].apply(lambda x: df.loc[x, 'Long'])
test_df_pop['Lat'] = test_df_pop['country+province'].apply(lambda x: df.loc[x, 'Lat'])
test_df_pop['Long'] = test_df_pop['country+province'].apply(lambda x: df.loc[x, 'Long'])
mo = train_df_pop['Date'].apply(lambda x: x[5:7])
da = train_df_pop['Date'].apply(lambda x: x[8:10])
mo_test = test_df_pop['Date'].apply(lambda x: x[5:7])
da_test = test_df_pop['Date'].apply(lambda x: x[8:10])
train_df_pop['day_from_jan_first'] = (da.apply(int)
                               + 31*(mo=='02') 
                               + 60*(mo=='03')
                               + 91*(mo=='04')  
                              )
test_df_pop['day_from_jan_first'] = (da_test.apply(int)
                               + 31*(mo_test=='02') 
                               + 60*(mo_test=='03')
                               + 91*(mo_test=='04')  
                              )

In [32]:
C = []
for j in train_df_pop.index:
    df_train = train_df_pop.iloc[j:(j+1)]
    mat = cdist(df_train[['Lat','Long', 'day_from_jan_first']],
                weather_df[['lat','lon', 'day_from_jan_first']], 
                metric='euclidean')
    new_df = pd.DataFrame(mat, index=df_train.Id, columns=weather_df.index)
    arr = new_df.values
    new_close = np.where(arr == np.nanmin(arr, axis=1)[:,None],new_df.columns,False)
    L = [i[i.astype(bool)].tolist()[0] for i in new_close]
    C.append(L[0])
    
train_df_pop['closest_station'] = C
train_df_pop= train_df_pop.set_index('closest_station').join(weather_df[['temp', 'min', 'max', 'stp', 'wdsp', 'prcp', 'fog']], ).reset_index().drop(['index'], axis=1)
train_df_pop.sort_values(by=['Id'], inplace=True)
train_df_pop.index = train_df_pop['Id'].apply(lambda x: x-1)

In [33]:
D=[]

for j in test_df_pop.index:
    df_test = test_df_pop.iloc[j:(j+1)]
    mat_test = cdist(df_test[['Lat','Long', 'day_from_jan_first']],
                weather_df[['lat','lon', 'day_from_jan_first']], 
                metric='euclidean')
    new_df_test = pd.DataFrame(mat_test, index=df_test.ForecastId, columns=weather_df.index)
    arr_test = new_df_test.values
    new_close_test = np.where(arr_test == np.nanmin(arr_test, axis=1)[:,None],new_df_test.columns,False)
    L_test = [i[i.astype(bool)].tolist()[0] for i in new_close_test]
    D.append(L_test[0])
    
test_df_pop['closest_station'] = D
test_df_pop= test_df_pop.set_index('closest_station').join(weather_df[['temp', 'min', 'max', 'stp', 'wdsp', 'prcp', 'fog']], ).reset_index().drop(['index'], axis=1)
test_df_pop.sort_values(by=['ForecastId'], inplace=True)
test_df_pop.index = test_df_pop['ForecastId'].apply(lambda x: x-1)

In [34]:
train_df_pop.reset_index(drop=True, inplace=True)

In [35]:
train_df_pop["Id"]=train_df_pop.index

In [36]:
train_df_pop.isnull().sum()

Id                       0
Province_State        9633
Country_Region           0
Date                     0
ConfirmedCases           0
Fatalities               0
Population (2020)        0
Density                  0
country+province         0
Lat                      0
Long                     0
day_from_jan_first       0
temp                     0
min                      0
max                      0
stp                      0
wdsp                     0
prcp                     0
fog                      0
dtype: int64

In [37]:
labelencoder = LabelEncoder()

In [38]:
train_df_pop['Country_Region_Types'] = labelencoder.fit_transform(train_df_pop['Country_Region'])

In [39]:
test_df_pop['Country_Region_Types'] = labelencoder.fit_transform(test_df_pop['Country_Region'])

In [40]:
train_df_pop.head()

,Id,Province_State,Country_Region,Date,ConfirmedCases,Fatalities,Population (2020),Density,country+province,Lat,Long,day_from_jan_first,temp,min,max,stp,wdsp,prcp,fog,Country_Region_Types
0,0,NaN,Afghanistan,2020-01-22,0.0,0.0,38928346.0,60.0,Afghanistan-,33.0,65.0,22,42.6,33.6,54.9,999.9,9.4,0.00,0,0
1,1,NaN,Afghanistan,2020-01-23,0.0,0.0,38928346.0,60.0,Afghanistan-,33.0,65.0,23,42.0,32.7,55.9,999.9,14.9,99.99,1,0
2,2,NaN,Afghanistan,2020-01-24,0.0,0.0,38928346.0,60.0,Afghanistan-,33.0,65.0,24,40.1,36.9,43.2,999.9,10.4,0.17,1,0
3,3,NaN,Afghanistan,2020-01-25,0.0,0.0,38928346.0,60.0,Afghanistan-,33.0,65.0,25,46.0,37.9,56.3,999.9,6.1,0.57,1,0
4,4,NaN,Afghanistan,2020-01-26,0.0,0.0,38928346.0,60.0,Afghanistan-,33.0,65.0,26,42.8,36.1,53.1,999.9,10.8,0.00,1,0


In [41]:
test_df_pop.head()

,ForecastId,Province_State,Country_Region,Date,Population (2020),Density,country+province,Lat,Long,day_from_jan_first,temp,min,max,stp,wdsp,prcp,fog,Country_Region_Types
ForecastId,,,,,,,,,,,,,,,,,,
0,1,NaN,Afghanistan,2020-03-19,38928346.0,60.0,Afghanistan-,33.0,65.0,79,45.2,32.0,59.7,775.3,5.3,0.31,1,0
1,2,NaN,Afghanistan,2020-03-20,38928346.0,60.0,Afghanistan-,33.0,65.0,80,39.4,32.7,43.5,773.2,4.8,1.22,1,0
2,3,NaN,Afghanistan,2020-03-21,38928346.0,60.0,Afghanistan-,33.0,65.0,81,42.4,34.7,54.3,776.3,2.7,2.20,1,0
3,4,NaN,Afghanistan,2020-03-22,38928346.0,60.0,Afghanistan-,33.0,65.0,82,44.7,35.8,53.8,774.7,7.5,0.00,1,0
4,5,NaN,Afghanistan,2020-03-23,38928346.0,60.0,Afghanistan-,33.0,65.0,83,45.0,37.9,50.4,769.8,7.3,0.47,1,0


In [42]:
train_df_pop.rename(columns={"Population (2020)":"Population"}, inplace=True)

In [43]:
test_df_pop.rename(columns={"Population (2020)":"Population"}, inplace=True)

In [44]:
train_df_pop.head()

,Id,Province_State,Country_Region,Date,ConfirmedCases,Fatalities,Population,Density,country+province,Lat,Long,day_from_jan_first,temp,min,max,stp,wdsp,prcp,fog,Country_Region_Types
0,0,NaN,Afghanistan,2020-01-22,0.0,0.0,38928346.0,60.0,Afghanistan-,33.0,65.0,22,42.6,33.6,54.9,999.9,9.4,0.00,0,0
1,1,NaN,Afghanistan,2020-01-23,0.0,0.0,38928346.0,60.0,Afghanistan-,33.0,65.0,23,42.0,32.7,55.9,999.9,14.9,99.99,1,0
2,2,NaN,Afghanistan,2020-01-24,0.0,0.0,38928346.0,60.0,Afghanistan-,33.0,65.0,24,40.1,36.9,43.2,999.9,10.4,0.17,1,0
3,3,NaN,Afghanistan,2020-01-25,0.0,0.0,38928346.0,60.0,Afghanistan-,33.0,65.0,25,46.0,37.9,56.3,999.9,6.1,0.57,1,0
4,4,NaN,Afghanistan,2020-01-26,0.0,0.0,38928346.0,60.0,Afghanistan-,33.0,65.0,26,42.8,36.1,53.1,999.9,10.8,0.00,1,0


In [45]:
test_df_pop.head()

,ForecastId,Province_State,Country_Region,Date,Population,Density,country+province,Lat,Long,day_from_jan_first,temp,min,max,stp,wdsp,prcp,fog,Country_Region_Types
ForecastId,,,,,,,,,,,,,,,,,,
0,1,NaN,Afghanistan,2020-03-19,38928346.0,60.0,Afghanistan-,33.0,65.0,79,45.2,32.0,59.7,775.3,5.3,0.31,1,0
1,2,NaN,Afghanistan,2020-03-20,38928346.0,60.0,Afghanistan-,33.0,65.0,80,39.4,32.7,43.5,773.2,4.8,1.22,1,0
2,3,NaN,Afghanistan,2020-03-21,38928346.0,60.0,Afghanistan-,33.0,65.0,81,42.4,34.7,54.3,776.3,2.7,2.20,1,0
3,4,NaN,Afghanistan,2020-03-22,38928346.0,60.0,Afghanistan-,33.0,65.0,82,44.7,35.8,53.8,774.7,7.5,0.00,1,0
4,5,NaN,Afghanistan,2020-03-23,38928346.0,60.0,Afghanistan-,33.0,65.0,83,45.0,37.9,50.4,769.8,7.3,0.47,1,0


In [46]:
X_dataset=train_df_pop.copy()

In [47]:
X_dataset=X_dataset[["Date","Population","Density","day_from_jan_first","temp","min","max","stp","wdsp","prcp","fog","Country_Region_Types","Lat","Long"]]

In [48]:
X_dataset["Date"] = X_dataset["Date"].apply(lambda x:x.replace("-",""))
X_dataset["Date"]  = X_dataset["Date"].astype(int)

In [49]:
X_dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16758 entries, 0 to 16757
Data columns (total 14 columns):
Date                    16758 non-null int64
Population              16758 non-null float64
Density                 16758 non-null float64
day_from_jan_first      16758 non-null int64
temp                    16758 non-null float64
min                     16758 non-null float64
max                     16758 non-null float64
stp                     16758 non-null float64
wdsp                    16758 non-null object
prcp                    16758 non-null float64
fog                     16758 non-null object
Country_Region_Types    16758 non-null int64
Lat                     16758 non-null float64
Long                    16758 non-null float64
dtypes: float64(9), int64(3), object(2)
memory usage: 1.8+ MB


In [50]:
X_dataset.head()

,Date,Population,Density,day_from_jan_first,temp,min,max,stp,wdsp,prcp,fog,Country_Region_Types,Lat,Long
0,20200122,38928346.0,60.0,22,42.6,33.6,54.9,999.9,9.4,0.00,0,0,33.0,65.0
1,20200123,38928346.0,60.0,23,42.0,32.7,55.9,999.9,14.9,99.99,1,0,33.0,65.0
2,20200124,38928346.0,60.0,24,40.1,36.9,43.2,999.9,10.4,0.17,1,0,33.0,65.0
3,20200125,38928346.0,60.0,25,46.0,37.9,56.3,999.9,6.1,0.57,1,0,33.0,65.0
4,20200126,38928346.0,60.0,26,42.8,36.1,53.1,999.9,10.8,0.00,1,0,33.0,65.0


In [51]:
X_dataset["fog"]  = X_dataset["fog"].astype(int)
X_dataset["wdsp"]  = X_dataset["wdsp"].astype(float)

In [52]:
X_dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16758 entries, 0 to 16757
Data columns (total 14 columns):
Date                    16758 non-null int64
Population              16758 non-null float64
Density                 16758 non-null float64
day_from_jan_first      16758 non-null int64
temp                    16758 non-null float64
min                     16758 non-null float64
max                     16758 non-null float64
stp                     16758 non-null float64
wdsp                    16758 non-null float64
prcp                    16758 non-null float64
fog                     16758 non-null int64
Country_Region_Types    16758 non-null int64
Lat                     16758 non-null float64
Long                    16758 non-null float64
dtypes: float64(10), int64(4)
memory usage: 1.8 MB


In [53]:
X_dataset["Date"].max()

20200318

In [54]:
y1=train_df_pop[train_df_pop["Date"]<"2020-03-19"]["ConfirmedCases"]
y2=train_df_pop[train_df_pop["Date"]<"2020-03-19"]["Fatalities"]

In [55]:
#Confirmed Cases
X_train_confirmed, X_test_confirmed, y_train_confirmed, y_test_confirmed = train_test_split(X_dataset, y1, test_size = .20, random_state = 42)

In [56]:
dt1=DecisionTreeRegressor(criterion="friedman_mse",max_depth=20,random_state=42)

In [57]:
dt1.fit(X_train_confirmed, y_train_confirmed)

DecisionTreeRegressor(ccp_alpha=0.0, criterion='friedman_mse', max_depth=20,
                      max_features=None, max_leaf_nodes=None,
                      min_impurity_decrease=0.0, min_impurity_split=None,
                      min_samples_leaf=1, min_samples_split=2,
                      min_weight_fraction_leaf=0.0, presort='deprecated',
                      random_state=42, splitter='best')

In [58]:
y_pred_dt_confirmed=dt1.predict(X_test_confirmed)

In [59]:
np.sqrt(mean_squared_log_error( y_test_confirmed, y_pred_dt_confirmed ))

0.5561725137130228

In [60]:
#Fatalities
X_train_fatal, X_test_fatal, y_train_fatal, y_test_fatal = train_test_split(X_dataset, y2, test_size = .20, random_state = 42)

In [61]:
dt2=DecisionTreeRegressor(criterion="friedman_mse",max_depth=20,random_state=42)

In [62]:
dt2.fit(X_train_fatal, y_train_fatal)

DecisionTreeRegressor(ccp_alpha=0.0, criterion='friedman_mse', max_depth=20,
                      max_features=None, max_leaf_nodes=None,
                      min_impurity_decrease=0.0, min_impurity_split=None,
                      min_samples_leaf=1, min_samples_split=2,
                      min_weight_fraction_leaf=0.0, presort='deprecated',
                      random_state=42, splitter='best')

In [63]:
y_pred_dt_fatal=dt2.predict(X_test_fatal)

In [64]:
np.sqrt(mean_squared_log_error( y_test_fatal, y_pred_dt_fatal ))

0.32442159308122837

In [65]:
test_data=test_df_pop[["Date","Population","Density","day_from_jan_first","temp","min","max","stp","wdsp","prcp","fog","Country_Region_Types","Lat","Long"]]

In [66]:
test_data.head()

,Date,Population,Density,day_from_jan_first,temp,min,max,stp,wdsp,prcp,fog,Country_Region_Types,Lat,Long
ForecastId,,,,,,,,,,,,,,
0,2020-03-19,38928346.0,60.0,79,45.2,32.0,59.7,775.3,5.3,0.31,1,0,33.0,65.0
1,2020-03-20,38928346.0,60.0,80,39.4,32.7,43.5,773.2,4.8,1.22,1,0,33.0,65.0
2,2020-03-21,38928346.0,60.0,81,42.4,34.7,54.3,776.3,2.7,2.20,1,0,33.0,65.0
3,2020-03-22,38928346.0,60.0,82,44.7,35.8,53.8,774.7,7.5,0.00,1,0,33.0,65.0
4,2020-03-23,38928346.0,60.0,83,45.0,37.9,50.4,769.8,7.3,0.47,1,0,33.0,65.0


In [67]:
test_data["Date"] = test_data["Date"].apply(lambda x:x.replace("-",""))
test_data["Date"]  = test_data["Date"].astype(int)

/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [68]:
y_confirmed=dt1.predict(test_data)

In [69]:
y_fatal=dt2.predict(test_data)

In [70]:
submission=pd.DataFrame({'ForecastId': test_df["ForecastId"], 'ConfirmedCases': y_confirmed, 'Fatalities': y_fatal})

In [71]:
submission["ConfirmedCases"]=submission["ConfirmedCases"].astype(int)
submission["Fatalities"]=submission["Fatalities"].astype(int)

In [72]:
submission.head()

,ForecastId,ConfirmedCases,Fatalities
0,1,22,0
1,2,38,0
2,3,22,0
3,4,169,0
4,5,169,0


In [73]:
submission.to_csv('submission.csv', index=False)